# Полезные методы

Рассмотрим некоторые полезные методы для работы с данными в Pandas. Дополнительные методы можно изучить в официальной
[документации](https://pandas.pydata.org/pandas-docs/stable/reference/index.html).

В Данном блокноте будут рассмотрены следующие методы:
* [метод apply()](#apply_method)
* [apply() с функцией](#apply_function)
* [apply() с лямбда-выражением](#apply_lambda)
* [apply() для нескольких колонок](#apply_multiple)
* [describe()](#describe)
* [sort_values()](#sort)
* [idmin и idmax](#idx)
* [corr()](#corr)
* [value_counts](#v_c)
* [unique и nunique](#uni)
* [replace](#replace)
* [map](#map)
* [duplicated и drop_duplicates](#dup)
* [between](#bet)
* [nlargest и nsmallest](#n)
* [sample](#sample)

<a id='apply_method'></a>
## Метод .apply()

Данный метод позволяет применить ту или иную функцию сразу для всех значений колонки DataFrame (в режиме broadcast).

In [10]:
import numpy as np
import pandas as pd

In [12]:
df = pd.read_csv('tips.csv')
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458
3,23.68,3.31,Male,No,Sun,Dinner,2,11.84,Nathaniel Harris,4676137647685994,Sun5260
4,24.59,3.61,Female,No,Sun,Dinner,4,6.15,Tonya Carter,4832732618637221,Sun2251


<a id='apply_function'></a>
### Метод apply() с функцией

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   total_bill        244 non-null    float64
 1   tip               244 non-null    float64
 2   sex               244 non-null    object 
 3   smoker            244 non-null    object 
 4   day               244 non-null    object 
 5   time              244 non-null    object 
 6   size              244 non-null    int64  
 7   price_per_person  244 non-null    float64
 8   Payer Name        244 non-null    object 
 9   CC Number         244 non-null    int64  
 10  Payment ID        244 non-null    object 
dtypes: float64(3), int64(2), object(6)
memory usage: 21.1+ KB


In [31]:
def last_four(num):
    """Функция, которая приводит число к строке и возвращает последние 4 символа."""
    return str(num)[-4:]

In [35]:
last_four(654980650)

'0650'

In [39]:
# Применение написанной функции ко всем значениям колонки из датафрейма
# Так как мы не запскаем функцию, а передаем ее в качестве параметра, то last_four без скобок
df['CC Number'].apply(last_four)

0      3410
1      9230
2      1322
3      5994
4      7221
       ... 
239    2842
240    5404
241    7196
242    0950
243    8139
Name: CC Number, Length: 244, dtype: object

In [41]:
# Можно добавить как новую колонку в исходный датафрейм
df['last_four'] = df['CC Number'].apply(last_four)
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,last_four
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959,3410
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608,9230
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458,1322
3,23.68,3.31,Male,No,Sun,Dinner,2,11.84,Nathaniel Harris,4676137647685994,Sun5260,5994
4,24.59,3.61,Female,No,Sun,Dinner,4,6.15,Tonya Carter,4832732618637221,Sun2251,7221


### Применение метода apply() для более сложных функций

In [44]:
# Функция для присуждения рейтинг дороговизны ресторану
def yelp(price):
    if price < 10:
        return '$'
    elif price >= 10 and price < 30:
        return '$$'
    else:
        return '$$$'

In [46]:
df['yelp'] = df['total_bill'].apply(yelp)
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,last_four,yelp
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959,3410,$$
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608,9230,$$
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458,1322,$$
3,23.68,3.31,Male,No,Sun,Dinner,2,11.84,Nathaniel Harris,4676137647685994,Sun5260,5994,$$
4,24.59,3.61,Female,No,Sun,Dinner,4,6.15,Tonya Carter,4832732618637221,Sun2251,7221,$$


In [48]:
df['yelp']

0      $$
1      $$
2      $$
3      $$
4      $$
       ..
239    $$
240    $$
241    $$
242    $$
243    $$
Name: yelp, Length: 244, dtype: object

*__Создаваемые функции должны возвращать только одно значение!!! Функция не должна возвращать объект Series, т.к. 
Pandas сам будет создавать объект Series, применяя функцию ко всем строкам того объекта Series, к которому применяется
метод apply()__*

<a id='apply_lambda'></a>
## Метод apply() с лямбда-выражением

Eсть несколько различных способов применить apply для нескольких колонок:

https://stackoverflow.com/questions/19914937/applying-function-with-multiple-arguments-to-create-a-new-pandas-column

In [57]:
# lambda - функция без названия, которая применяется один раз
# (название входной переменной может быть любым)
lambda num: num*2

<function __main__.<lambda>(num)>

In [55]:
df['total_bill'].apply(lambda num: num*2)

0      33.98
1      20.68
2      42.02
3      47.36
4      49.18
       ...  
239    58.06
240    54.36
241    45.34
242    35.64
243    37.56
Name: total_bill, Length: 244, dtype: float64

<a id='apply_multiple'></a>
## Метод apply() с несколькими колонками

In [62]:
def quality(total_bill, tip):
    """Функция для определения типа чаевых (обычные/щедрые)."""
    if tip / total_bill > 0.25:
        return 'Щедрые чаевые'
    else:
        return 'Обычные чаевые'

In [64]:
quality(16.99, 1.01)

'Обычные чаевые'

### Форматирование с помощью lambda-выражения

1. Для начала передаем список требуемых колонок __df[['total_bill', 'tip']]__
2. После этого импользуем метод __apply__, в котором будет применяться __lambda-выражение__, которое на вход принимает
исходный датафрейм __df__, а на выходе возвращает функцию __quality__, для которой в параметрах будут указаны два парметра
df['total_bill'] и df['tip']
3. Это выражение будет применяться вдоль оси axis=1, это нужно для того, чтобы сообщить, что мы ссылаемся на колонки. То есть,
axis=1 является не параметром lambda-выражения, а вторым параметром метода apply (первым является как раз lambda-выражение).

In [67]:
df[['total_bill','tip']].apply(lambda df: quality(df['total_bill'], df['tip']), axis=1)

0      Обычные чаевые
1      Обычные чаевые
2      Обычные чаевые
3      Обычные чаевые
4      Обычные чаевые
            ...      
239    Обычные чаевые
240    Обычные чаевые
241    Обычные чаевые
242    Обычные чаевые
243    Обычные чаевые
Length: 244, dtype: object

In [69]:
# Если необходимо сохранить полученный результат.
df['quality'] = df[['total_bill','tip']].apply(lambda df: quality(df['total_bill'], df['tip']), axis=1)
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,last_four,yelp,quality
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959,3410,$$,Обычные чаевые
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608,9230,$$,Обычные чаевые
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458,1322,$$,Обычные чаевые
3,23.68,3.31,Male,No,Sun,Dinner,2,11.84,Nathaniel Harris,4676137647685994,Sun5260,5994,$$,Обычные чаевые
4,24.59,3.61,Female,No,Sun,Dinner,4,6.15,Tonya Carter,4832732618637221,Sun2251,7221,$$,Обычные чаевые


In [71]:
# Можно сделать все то же самое, без использования дополнительной функции quality()
df[['total_bill', 'tip']].apply(lambda df: "Щедрые чаевые" if (df['tip'] / df['total_bill']) > 0.25 else "Обычные чаевые", axis=1)

0      Обычные чаевые
1      Обычные чаевые
2      Обычные чаевые
3      Обычные чаевые
4      Обычные чаевые
            ...      
239    Обычные чаевые
240    Обычные чаевые
241    Обычные чаевые
242    Обычные чаевые
243    Обычные чаевые
Length: 244, dtype: object

### Как ускорить подобные операции с помощью  np.vectorize(), и упростить синтаксис

np.vectorize() нужен для того, чтобы преобразовать функции, которые не предназначены для работы с numpy

Детальное описание vectorize: https://docs.scipy.org/doc/numpy/reference/generated/numpy.vectorize.html

1. Векторизуется функция quality
2. Передаются колонки

In [76]:
df['quality'] = np.vectorize(quality)(df['total_bill'], df['tip'])
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,last_four,yelp,quality
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959,3410,$$,Обычные чаевые
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608,9230,$$,Обычные чаевые
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458,1322,$$,Обычные чаевые
3,23.68,3.31,Male,No,Sun,Dinner,2,11.84,Nathaniel Harris,4676137647685994,Sun5260,5994,$$,Обычные чаевые
4,24.59,3.61,Female,No,Sun,Dinner,4,6.15,Tonya Carter,4832732618637221,Sun2251,7221,$$,Обычные чаевые


Проверим какой из способов быстрее

In [80]:
import timeit

Библиотеке timeit на вход подаются фрагменты кода в виде текста. Сначала - выполняется один раз команда
setup, а затем выполняются те команды, для которых необходимо замерить время выполнения. Они будут выполняться 
много раз в цикле и на выходе получится время выполнения этих команд.

In [82]:
# этот фрагмент кода будет запускаться только один раз
setup = '''
import numpy as np
import pandas as pd
df = pd.read_csv('tips.csv')
def quality(total_bill,tip):
    if tip/total_bill  > 0.25:
        return "Generous"
    else:
        return "Other"
'''

In [84]:
# это фрагменты кода, для которых будет измеряться время выполнения
# (они будут выполняться много раз)
stmt_one = ''' 
df['Tip Quality'] = df[['total_bill','tip']].apply(lambda df: quality(df['total_bill'],df['tip']),axis=1)
'''

stmt_two = '''
df['Tip Quality'] = np.vectorize(quality)(df['total_bill'], df['tip'])
'''

In [87]:
# setup - будет вызван один раз, а stmt_one - тысячу раз
timeit.timeit(setup=setup, stmt=stmt_one, number=1000)

1.1685118000004877

Это значение для lambda-выражения

In [90]:
timeit.timeit(setup=setup, stmt=stmt_two, number=1000)

0.13251779999882274

Это значение для np.vectorize()

<a id='describe'></a>
## Метод df.describe() для суммарных статистик

Метод выводит основные характеристики для числовых колонок: среднее значение, максимальное, минимальное и т.д.

In [101]:
df = pd.read_csv('tips.csv')

In [104]:
df.describe()

,total_bill,tip,size,price_per_person,CC Number
count,244.000000,244.000000,244.000000,244.000000,2.440000e+02
mean,19.785943,2.998279,2.569672,7.888197,2.563496e+15
std,8.902412,1.383638,0.951100,2.914234,2.369340e+15
min,3.070000,1.000000,1.000000,2.880000,6.040679e+10
25%,13.347500,2.000000,2.000000,5.800000,3.040731e+13
50%,17.795000,2.900000,2.000000,7.255000,3.525318e+15
75%,24.127500,3.562500,3.000000,9.390000,4.553675e+15
max,50.810000,10.000000,6.000000,20.270000,6.596454e+15


In [106]:
# Можно выполнить транспонирование таблицы, для улучшения восприятия данных
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
total_bill,244.0,1.978594e+01,8.902412e+00,3.070000e+00,1.334750e+01,1.779500e+01,2.412750e+01,5.081000e+01
tip,244.0,2.998279e+00,1.383638e+00,1.000000e+00,2.000000e+00,2.900000e+00,3.562500e+00,1.000000e+01
size,244.0,2.569672e+00,9.510998e-01,1.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,6.000000e+00
price_per_person,244.0,7.888197e+00,2.914234e+00,2.880000e+00,5.800000e+00,7.255000e+00,9.390000e+00,2.027000e+01
CC Number,244.0,2.563496e+15,2.369340e+15,6.040679e+10,3.040731e+13,3.525318e+15,4.553675e+15,6.596454e+15


<a id='sort'></a>
## Метод .sort_values()

Когда необходимо отсортировать данные в датафрейме

__Изменения в исходный датафрейм не вносятся, необходимо явное сохранение!__

In [129]:
# Сортировка по колонке от самого малого значения, к самому большому
df.sort_values('tip')

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
67,3.07,1.00,Female,Yes,Sat,Dinner,1,3.07,Tiffany Brock,4359488526995267,Sat3455
236,12.60,1.00,Male,Yes,Sat,Dinner,2,6.30,Matthew Myers,3543676378973965,Sat5032
92,5.75,1.00,Female,Yes,Fri,Dinner,2,2.88,Leah Ramirez,3508911676966392,Fri3780
111,7.25,1.00,Female,No,Sat,Dinner,1,7.25,Terri Jones,3559221007826887,Sat4801
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959
...,...,...,...,...,...,...,...,...,...,...,...
141,34.30,6.70,Male,No,Thur,Lunch,6,5.72,Steven Carlson,3526515703718508,Thur1025
59,48.27,6.73,Male,No,Sat,Dinner,4,12.07,Brian Ortiz,6596453823950595,Sat8139
23,39.42,7.58,Male,No,Sat,Dinner,4,9.86,Lance Peterson,3542584061609808,Sat239
212,48.33,9.00,Male,No,Sat,Dinner,4,12.08,Alex Williamson,676218815212,Sat4590


In [131]:
# Обратная сортировка, от самого большого, к самому малому 
df.sort_values('tip', ascending=False)

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
170,50.81,10.00,Male,Yes,Sat,Dinner,3,16.94,Gregory Clark,5473850968388236,Sat1954
212,48.33,9.00,Male,No,Sat,Dinner,4,12.08,Alex Williamson,676218815212,Sat4590
23,39.42,7.58,Male,No,Sat,Dinner,4,9.86,Lance Peterson,3542584061609808,Sat239
59,48.27,6.73,Male,No,Sat,Dinner,4,12.07,Brian Ortiz,6596453823950595,Sat8139
141,34.30,6.70,Male,No,Thur,Lunch,6,5.72,Steven Carlson,3526515703718508,Thur1025
...,...,...,...,...,...,...,...,...,...,...,...
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959
236,12.60,1.00,Male,Yes,Sat,Dinner,2,6.30,Matthew Myers,3543676378973965,Sat5032
111,7.25,1.00,Female,No,Sat,Dinner,1,7.25,Terri Jones,3559221007826887,Sat4801
67,3.07,1.00,Female,Yes,Sat,Dinner,1,3.07,Tiffany Brock,4359488526995267,Sat3455


In [136]:
# Можно сортировать по нескольким колонкам сразу (передаем список колонок)
df.sort_values(['tip', 'size'], ascending=True)

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
67,3.07,1.00,Female,Yes,Sat,Dinner,1,3.07,Tiffany Brock,4359488526995267,Sat3455
111,7.25,1.00,Female,No,Sat,Dinner,1,7.25,Terri Jones,3559221007826887,Sat4801
92,5.75,1.00,Female,Yes,Fri,Dinner,2,2.88,Leah Ramirez,3508911676966392,Fri3780
236,12.60,1.00,Male,Yes,Sat,Dinner,2,6.30,Matthew Myers,3543676378973965,Sat5032
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959
...,...,...,...,...,...,...,...,...,...,...,...
141,34.30,6.70,Male,No,Thur,Lunch,6,5.72,Steven Carlson,3526515703718508,Thur1025
59,48.27,6.73,Male,No,Sat,Dinner,4,12.07,Brian Ortiz,6596453823950595,Sat8139
23,39.42,7.58,Male,No,Sat,Dinner,4,9.86,Lance Peterson,3542584061609808,Sat239
212,48.33,9.00,Male,No,Sat,Dinner,4,12.08,Alex Williamson,676218815212,Sat4590


Данные сначала отсортировались по колонке __'tip'__, а затем для тех строк где встречается одно и то же значение
колонки __'tip'__, происходит сортировка по следующей колонке __'size'__.

<a id='idx'></a>
## Методы idmin() и idmax()

Нахождение позиции индекса, на которой встречались минимальное и максимальное значение

In [144]:
# Нахождение максимального числа в колонке (объекте Series)
df['total_bill'].max()

50.81

In [146]:
# Нахождение индекса строки с масимальным значением
df['total_bill'].idxmax()

170

In [150]:
# Можно найти строку по полученному индексу
df.iloc[170]

total_bill                     50.81
tip                             10.0
sex                             Male
smoker                           Yes
day                              Sat
time                          Dinner
size                               3
price_per_person               16.94
Payer Name             Gregory Clark
CC Number           5473850968388236
Payment ID                   Sat1954
Name: 170, dtype: object

In [154]:
# Нахождение минимального числа в колонке (объекте Series)
df['total_bill'].min()

3.07

In [156]:
# Нахождение индекса строки с минимальным значением
df['total_bill'].idxmin()

67

In [158]:
df.iloc[67]

total_bill                      3.07
tip                              1.0
sex                           Female
smoker                           Yes
day                              Sat
time                          Dinner
size                               1
price_per_person                3.07
Payer Name             Tiffany Brock
CC Number           4359488526995267
Payment ID                   Sat3455
Name: 67, dtype: object

In [160]:
# Можно сделать все в одну строку
df.iloc[df['total_bill'].idxmin()]

total_bill                      3.07
tip                              1.0
sex                           Female
smoker                           Yes
day                              Sat
time                          Dinner
size                               1
price_per_person                3.07
Payer Name             Tiffany Brock
CC Number           4359488526995267
Payment ID                   Sat3455
Name: 67, dtype: object

<a id='corr'></a>
## Метод df.corr() для проверки корреляции

[Что такое корреляция](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D1%80%D1%80%D0%B5%D0%BB%D1%8F%D1%86%D0%B8%D1%8F)

В последних версиях Pandas - параметр для выбора только числовых колонок:  
__df.corr(numeric_only=True)__

In [166]:
df.corr(numeric_only=True)

,total_bill,tip,size,price_per_person,CC Number
total_bill,1.000000,0.675734,0.598315,0.647554,0.104576
tip,0.675734,1.000000,0.489299,0.347405,0.110857
size,0.598315,0.489299,1.000000,-0.175359,-0.030239
price_per_person,0.647554,0.347405,-0.175359,1.000000,0.135240
CC Number,0.104576,0.110857,-0.030239,0.135240,1.000000


In [168]:
# Для нескольких отдельных колонок
df[['total_bill', 'tip']].corr()

,total_bill,tip
total_bill,1.000000,0.675734
tip,0.675734,1.000000


<a id='v_c'></a>
## Метод df.value_counts() 

Удобный способ быстро посчитать количество строк для той или иной категории. Имеет смысл только для колонок, принимающих
[категорийные (категориальные/дискретные) значения](https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%87%D0%B5%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D0%B0%D1%8F_%D0%BF%D0%B5%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%BD%D0%B0%D1%8F).

In [176]:
# Например подсчет количества строк для мужчин и для женщин
df['sex'].value_counts()

sex
Male      157
Female     87
Name: count, dtype: int64

<a id='uni'></a>
## df.unique() 

Для анализа уникальности значений 

In [188]:
# Получение массива с уникальными значениями в колонке
df['day'].unique()

array(['Sun', 'Sat', 'Thur', 'Fri'], dtype=object)

In [190]:
# Количество уникальных значений в колонке
df['day'].nunique()

4

<a id='replace'></a>

## Метод для замены значений replace()

In [194]:
# Female на F
df['sex'].replace('Female', 'F')

0         F
1      Male
2      Male
3      Male
4         F
       ... 
239    Male
240       F
241    Male
242    Male
243       F
Name: sex, Length: 244, dtype: object

In [196]:
# Если надо заменить несколько значений - передаем два списка [старые_значения], [новые_значения]
df['sex'].replace(['Male', 'Female'], ['M', 'F'])

0      F
1      M
2      M
3      M
4      F
      ..
239    M
240    F
241    M
242    M
243    F
Name: sex, Length: 244, dtype: object

<a id='map'></a>

## Метод для замены значений map()

In [200]:
# Словарь, содержащие соответствия(маппинг) значений
my_map = {'Female':'F', 'Male':'M'}

In [202]:
df['sex'].map(my_map)

0      F
1      M
2      M
3      M
4      F
      ..
239    M
240    F
241    M
242    M
243    F
Name: sex, Length: 244, dtype: object

<a id='dup'></a>

## Дубликаты строк и работа с ними
### .duplicated() и .drop_duplicates()

.duplicated() возвращает True для строк-дубликатов. Если есть несколько одинаковых строк, то для первой строки будет False,
а для всех остальных True, потому что остальные - это дубликаты первой строки.

In [210]:
# Проверка наличия дубликатов строк
df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
239    False
240    False
241    False
242    False
243    False
Length: 244, dtype: bool

In [212]:
simple_df = pd.DataFrame([1, 2, 2, 2], ['a', 'b', 'c', 'd'])

In [214]:
simple_df.duplicated()

a    False
b    False
c     True
d     True
dtype: bool

In [216]:
# Удаление дубликатов строк
simple_df.drop_duplicates()

,0
a,1
b,2


<a id='bet'></a>

## Метод .between()

Позволяет понять находится ли значение колонки между двумя указанными числами.

Первый параметр - left - левая граница диапазона;  
Второй параметр - right - правая граница диапазона;  
Третий параметр - inclusive - включать ли границы диапазона. В последних версиях Pandas: inclusice='both'.   
inclusive{“both”, “neither”, “left”, “right”}

In [226]:
df['total_bill'].between(10, 20, inclusive='both')

0       True
1       True
2      False
3      False
4      False
       ...  
239    False
240    False
241    False
242     True
243     True
Name: total_bill, Length: 244, dtype: bool

In [228]:
# С помощью такого выражения можно отфильтровать строки в исходном датафрейме
df[df['total_bill'].between(10, 20, inclusive='both')]

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608
8,15.04,1.96,Male,No,Sun,Dinner,2,7.52,Joseph Mcdonald,3522866365840377,Sun6820
9,14.78,3.23,Male,No,Sun,Dinner,2,7.39,Jerome Abbott,3532124519049786,Sun3775
10,10.27,1.71,Male,No,Sun,Dinner,2,5.14,William Riley,566287581219,Sun2546
...,...,...,...,...,...,...,...,...,...,...,...
234,15.53,3.00,Male,Yes,Sat,Dinner,2,7.76,Tracy Douglas,4097938155941930,Sat7220
235,10.07,1.25,Male,No,Sat,Dinner,2,5.04,Sean Gonzalez,3534021246117605,Sat4615
236,12.60,1.00,Male,Yes,Sat,Dinner,2,6.30,Matthew Myers,3543676378973965,Sat5032
242,17.82,1.75,Male,No,Sat,Dinner,2,8.91,Dennis Dixon,4375220550950,Sat17


<a id='n'></a>

## Методы  .nlargest() и .nsmallest()

Метод .nlargest() - возвращает первые n строк отсортировав данные по указанной колонке по убыванию.
Метод .nsmallest() - возвращает первые n строк отсортировав данные по указанной колонке по возрастанию.

In [233]:
df.nlargest(10, 'tip')

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
170,50.81,10.00,Male,Yes,Sat,Dinner,3,16.94,Gregory Clark,5473850968388236,Sat1954
212,48.33,9.00,Male,No,Sat,Dinner,4,12.08,Alex Williamson,676218815212,Sat4590
23,39.42,7.58,Male,No,Sat,Dinner,4,9.86,Lance Peterson,3542584061609808,Sat239
59,48.27,6.73,Male,No,Sat,Dinner,4,12.07,Brian Ortiz,6596453823950595,Sat8139
141,34.30,6.70,Male,No,Thur,Lunch,6,5.72,Steven Carlson,3526515703718508,Thur1025
183,23.17,6.50,Male,Yes,Sun,Dinner,4,5.79,Dr. Michael James,4718501859162,Sun6059
214,28.17,6.50,Female,Yes,Sat,Dinner,3,9.39,Marissa Jackson,4922302538691962,Sat3374
47,32.40,6.00,Male,No,Sun,Dinner,4,8.10,James Barnes,3552002592874186,Sun9677
239,29.03,5.92,Male,No,Sat,Dinner,3,9.68,Michael Avila,5296068606052842,Sat2657
88,24.71,5.85,Male,No,Thur,Lunch,2,12.36,Roger Taylor,4410248629955,Thur9003


In [235]:
df.nsmallest(2, 'tip')

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
67,3.07,1.0,Female,Yes,Sat,Dinner,1,3.07,Tiffany Brock,4359488526995267,Sat3455
92,5.75,1.0,Female,Yes,Fri,Dinner,2,2.88,Leah Ramirez,3508911676966392,Fri3780


<a id='sample'></a>

## Метод  .sample()

Возвращает случайный семпл данных. Можно указать либо количество строк __n=__, либо долю от общего
количества строк  __frac=__.  
При повторном запуске команды будем получать другой набор строк.

In [245]:
# Получение пяти случайных строк из набора
df.sample(5)

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
174,16.82,4.00,Male,Yes,Sun,Dinner,2,8.41,Brian Miles,3586342145399277,Sun7621
124,12.48,2.52,Female,No,Thur,Lunch,2,6.24,Jordan Diaz,4472778228206399,Thur208
147,11.87,1.63,Female,No,Thur,Lunch,2,5.94,Annette Cunningham,675937746864,Thur4780
28,21.70,4.30,Male,No,Sat,Dinner,2,10.85,David Collier,5529694315416009,Sat3697
10,10.27,1.71,Male,No,Sun,Dinner,2,5.14,William Riley,566287581219,Sun2546


In [247]:
# Получение 10% строк из набора
df.sample(frac=0.1)

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
63,18.29,3.76,Male,Yes,Sat,Dinner,4,4.57,Chad Hart,580171498976,Sat4178
44,30.40,5.60,Male,No,Sun,Dinner,4,7.60,Todd Cooper,503846761263,Sun2274
219,30.14,3.09,Female,Yes,Sat,Dinner,4,7.54,Shelby House,502097403252,Sat8863
147,11.87,1.63,Female,No,Thur,Lunch,2,5.94,Annette Cunningham,675937746864,Thur4780
203,16.40,2.50,Female,Yes,Thur,Lunch,2,8.20,Toni Brooks,3582289985920239,Thur7770
232,11.61,3.39,Male,No,Sat,Dinner,2,5.80,James Taylor,6011482917327995,Sat2124
90,28.97,3.00,Male,Yes,Fri,Dinner,2,14.48,Daniel Mason,3597456900644078,Fri4175
217,11.59,1.50,Male,Yes,Sat,Dinner,2,5.80,Gary Orr,30324521283406,Sat8489
139,13.16,2.75,Female,No,Thur,Lunch,2,6.58,Lindsey Meyer,676239597203,Thur6245
67,3.07,1.00,Female,Yes,Sat,Dinner,1,3.07,Tiffany Brock,4359488526995267,Sat3455
